In [ ]:
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

### Data Loading

In [ ]:
# CHANGE ME
dataset_path = "../Dataset/Regional averages/regional_averages_2.json"
feature_name = "U"

In [ ]:
naca_numbers = ['maximum_camber', 'maximum_camber_position', 'maximum_thickness']

dataset = []
with open(dataset_path, 'r') as dataset_file:
  samples = json.load(dataset_file)
  for sample in samples:
    dataset.append({
        "features": sample["features"][feature_name],
        "labels": list(sample["naca_numbers"].values())
    })

### Shuffling the dataset

In [ ]:
# Shuffling the dataset
np.random.shuffle(dataset)

### Training and test set

In [ ]:
# Defining the training and test set splitting percentage
split_percentage = 0.8

# Computing the number of training samples according to the splitting percentage
num_training_samples = int(np.floor(split_percentage * len(dataset)))

# Extracting the training and test set
training_set, test_set = dataset[:num_training_samples], dataset[num_training_samples:]

In [ ]:
# Extracting the training features and labels
train_features = np.array([sample["features"] for sample in training_set])
train_labels = np.array([sample["labels"] for sample in training_set])

# Extracting the test features and labels
test_features = np.array([sample["features"] for sample in test_set])
test_labels = np.array([sample["labels"] for sample in test_set])

### Data normalization

In [ ]:
# Computing the mean and standard deviation of the training features
mean = train_features.mean(axis=0)
std = train_features.std(axis=0)

In [ ]:
# Normalizing the training and test features w.r.t. the training statistics
normalized_train_features = (train_features - mean) / std
normalized_test_features = (test_features - mean) / std

In [ ]:
# Plotting a random sample
plt.plot(normalized_train_features[random.choice([0, len(normalized_train_features)-1])])
plt.show()

### Building the model

In [ ]:
# BEST MODEL FOR REGIONAL AVERAGES
def buildModel1():
  # Sequential Model
  model = keras.Sequential([
    keras.layers.InputLayer(input_shape=[np.shape(normalized_train_features)[1]]),
    keras.layers.Dropout(0.01),
    keras.layers.Dense(176, activation=tf.nn.relu),
    keras.layers.Dropout(0.01),
    keras.layers.Dense(208, activation=tf.nn.relu),
    keras.layers.Dropout(0.01),
    keras.layers.Dense(240, activation=tf.nn.relu),
    keras.layers.Dropout(0.01),
    keras.layers.Dense(240, activation=tf.nn.relu),
    keras.layers.Dropout(0.01),
    keras.layers.Dense(len(naca_numbers))
  ])

  # Compiling the model
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  
  return model

In [ ]:
# BEST MODEL FOR ARRIVAL TIMES
def buildModel2():
  # Sequential Model
  model = keras.Sequential([
    keras.layers.InputLayer(input_shape=[np.shape(normalized_train_features)[1]]),
    keras.layers.Dropout(0.03),
    keras.layers.Dense(96, activation=tf.nn.relu),
    keras.layers.Dropout(0.03),
    keras.layers.Dense(80, activation=tf.nn.relu),
    keras.layers.Dropout(0.03),
    keras.layers.Dense(48, activation=tf.nn.relu),
    keras.layers.Dropout(0.03),
    keras.layers.Dense(224, activation=tf.nn.relu),
    keras.layers.Dropout(0.03),
    keras.layers.Dense(len(naca_numbers))
  ])

  # Compiling the model
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  
  return model

In [ ]:
# BEST MODEL FOR REGIONAL ARRIVAL TIMES
def buildModel3():
  # Sequential Model
  model = keras.Sequential([
    keras.layers.InputLayer(input_shape=[np.shape(normalized_train_features)[1]]),
    keras.layers.Dropout(0.01),
    keras.layers.Dense(240, activation=tf.nn.relu),
    keras.layers.Dropout(0.01),
    keras.layers.Dense(80, activation=tf.nn.relu),
    keras.layers.Dropout(0.01),
    keras.layers.Dense(224, activation=tf.nn.relu),
    keras.layers.Dropout(0.01),
    keras.layers.Dense(len(naca_numbers))
  ])

  # Compiling the model
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  
  return model

In [ ]:
model = buildModel1()
model.summary()

### Model training

In [ ]:
# Defining the number of training epochs
epochs = 200

# Early stopping with a patience of 10 epochs
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Training the model
history = model.fit(
    normalized_train_features, 
    train_labels,
    epochs=epochs,
    validation_split = 0.2,
    verbose = 2,
    callbacks=[early_stopping]
)

In [ ]:
# Function to plot the metrics of training and validation
def plotHistory(history, training_metric, validation_metric, ylabel):
  plt.plot(history.history[training_metric], label=training_metric)
  plt.plot(history.history[validation_metric], label=validation_metric)
  plt.ylim([0, np.max(history.history[training_metric] + history.history[validation_metric])])
  plt.xlabel('Epoch')
  plt.ylabel(ylabel)
  plt.legend()
  plt.grid(True)
  plt.show()

In [ ]:
plotHistory(history, 'loss', 'val_loss', "Loss")
plotHistory(history, 'mae', 'val_mae', "Mean Absolute Error")

### Model evaluation

In [ ]:
# Computing the predictions of the test set
predictions = model.predict(normalized_test_features)

In [ ]:
# Function to compute the classification accuracy
def computeAccuracy():
    
    # Creating an array to save the results
    accuracy = np.zeros(len(naca_numbers))

    for idx in range(len(naca_numbers)):
        # Transform the NACA values to the same scale
        naca_predictions = predictions[:,idx] * 10 if idx == 1 else predictions[:,idx] * 100
        naca_labels = test_labels[:,idx] * 10 if idx == 1 else test_labels[:,idx] * 100

        # Converting the NACA values to the closest interger
        naca_predictions = np.array([round(prediction) for prediction in naca_predictions])
        naca_labels = np.array([round(label) for label in naca_labels])

        # Extracting the samples correctly classified
        correctly_classified = np.where(np.equal(naca_predictions, naca_labels))

        # Computing the classification accuracy of the current NACA number
        accuracy[idx] = np.shape(correctly_classified)[1] / len(naca_labels)

    return accuracy

In [ ]:
# Computing the classification accuracy
accuracy = computeAccuracy()

# Computing the regression errors: MSE and MAE
loss, mae = model.evaluate(normalized_test_features, test_labels, verbose = 0)

print("REGRESSION")
print(f" - Loss (Mean Square Error) --> {loss}")
print(f" - Mean Absolute Error --> {mae}")


print("\nCLASSIFICATION")
print(f" - Accuracy --> {np.mean(accuracy)}")
for i in range(len(naca_numbers)):
    print(f"   • {naca_numbers[i]} --> {accuracy[i]}")


In [ ]:
# Function to plot the predicted values
def plotPredictions(test_labels, test_predictions, label, color):
  plt.scatter(test_labels, test_predictions, label=label, color=color)
  plt.xlabel('True values')
  plt.ylabel('Predictions')
  plt.axis('equal')
  plt.axis('square')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.plot([0, 1], [0, 1], color="black")
  plt.legend()

In [ ]:
colors = ["blue", "green", "orange"]

# Plotting the obtained results
for i in range(len(naca_numbers)):
  plotPredictions(test_labels[:,i], predictions[:,i], label=naca_numbers[i], color=colors[i])